## Modeling *Ironsworn*’s core mechanic in [``dyce``](https://posita.github.io/dyce/)

Select ``Run All Cells`` from the ``Run`` menu above.

In [1]:
# Install additional requirements if necessary
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    try:
        import anydyce, tabulate
    except (ImportError, ModuleNotFoundError):
        requirements = ["anydyce~=0.4.0", "tabulate"]
        try:
            import piplite ; await piplite.install(requirements)
            # Work around <https://github.com/jupyterlite/jupyterlite/issues/838>
            import matplotlib.pyplot ; matplotlib.pyplot.clf()
        except ImportError:
            import pip ; pip.main(["install"] + requirements)
    import anydyce, tabulate

In [2]:
from dyce import H
from dyce.evaluation import HResult, PResult
from enum import IntEnum, auto

class IronSoloResult(IntEnum):
    SPECTACULAR_FAILURE = -1
    FAILURE = auto()
    WEAK_SUCCESS = auto()
    STRONG_SUCCESS = auto()
    SPECTACULAR_SUCCESS = auto()

d6 = H(6)
d10 = H(10)

def iron_solo_dependent_term(
    action: HResult,
    challenges: PResult,
    mod=0,
):
    modded_action = action.outcome + mod
    first_challenge_outcome, second_challenge_outcome = challenges.roll
    beats_first_challenge = modded_action > first_challenge_outcome
    beats_second_challenge = modded_action > second_challenge_outcome
    doubles = first_challenge_outcome == second_challenge_outcome
    if beats_first_challenge and beats_second_challenge:
        return (
            IronSoloResult.SPECTACULAR_SUCCESS
            if doubles
            else IronSoloResult.STRONG_SUCCESS
        )
    elif beats_first_challenge or beats_second_challenge:
        return IronSoloResult.WEAK_SUCCESS
    else:
        return (
            IronSoloResult.SPECTACULAR_FAILURE
            if doubles
            else IronSoloResult.FAILURE
        )

In [3]:
from dyce import P
from dyce.evaluation import foreach
from functools import partial
from IPython.display import display
from pandas import DataFrame, concat
import jinja2  # to appease the JupyterLite loader
import matplotlib, pandas

mods = list(range(0, 5))
df = pandas.DataFrame(columns=IronSoloResult)
results_by_mod = {}

for mod in mods:
    h_for_mod = foreach(
        partial(iron_solo_dependent_term, mod=mod),
        action=d6,
        challenges=2 @ P(d10),
    )
    results_by_mod[mod] = h_for_mod
    results_for_mod = dict(
        h_for_mod.zero_fill(IronSoloResult).distribution(rational_t=lambda n, d: n / d)
    )
    row = pandas.DataFrame(results_for_mod, columns=IronSoloResult, index=[mod])
    df = pandas.concat((df, row))

df.index.name = "Modifier"
# DataFrames use enum's values for displaying column names, so we convert them to
# names
df = df.rename(columns={v: v.name for v in IronSoloResult})
display(df)

,SPECTACULAR_FAILURE,FAILURE,WEAK_SUCCESS,STRONG_SUCCESS,SPECTACULAR_SUCCESS
Modifier,,,,,
0,0.075,0.516667,0.316667,0.066667,0.025
1,0.065,0.386667,0.396667,0.116667,0.035
2,0.055,0.276667,0.436667,0.186667,0.045
3,0.045,0.186667,0.436667,0.276667,0.055
4,0.035,0.116667,0.396667,0.386667,0.065


In [4]:
from anydyce import jupyter_visualize

jupyter_visualize(
    ((f"Modifier: {mod:+}", results_by_mod[mod]) for mod in mods),
    initial_burst_zero_fill_normalize=True,
)